# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

# Norvig's solution

In [1]:
import re
import nltk
import nltk
nltk.download('brown')
nltk.download('reuters')
nltk.download('gutenberg')
from nltk.corpus import brown, reuters, gutenberg
from tqdm import tqdm
import re
from collections import defaultdict
from tqdm import tqdm
import re
from collections import defaultdict, Counter
import math
import heapq
from functools import lru_cache
import re
from collections import Counter

[nltk_data] Downloading package brown to /usr/local/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
def words(text):
    return re.findall(r'\w+', text.lower())

brown_text = " ".join(brown.words())
reuters_text = " ".join(reuters.words())
gutenberg_text = " ".join(gutenberg.words())
all_text = brown_text + reuters_text + gutenberg_text

In [3]:
all_sentences = re.split(r'(?<=[.!?])\s+', all_text.strip())

In [4]:
import random

# ======= SPLIT DATA INTO TRAIN/TEST =======
random.seed(42)
random.shuffle(all_sentences)
split_index = int(0.95 * len(all_sentences))
train_sentences = all_sentences[:split_index]
test_sentences = all_sentences[split_index:]

In [5]:
train_text = " ".join(train_sentences)
train_words = words(train_text)

In [6]:
WORDS = Counter(train_words)

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [7]:
correction('speling')

'spelling'

In [8]:
def norvigs_correction(sentence):
    return " ".join([correction(word) for word in sentence.split()])

# Optimize Norvigs Solution with N-gram model

In [9]:
ngram_counts = {}

In [10]:
def build_ngram_counts(words, max_order):
    """
    Build n-gram counts for orders 1 through max_order.
    For unigrams, keys are one-element tuples.
    """
    global ngram_counts
    for i in range(max_order):
        if i + 1 not in ngram_counts:
            ngram_counts[i + 1] = {}

    # Order 1 (unigrams)
    for word in words:
        if ngram_counts[1].get((word,)) is None:
            ngram_counts[1][(word,)] = 1
        else:
            ngram_counts[1][(word,)] += 1
    
    # Orders 2 ... max_order
    for order in range(2, max_order + 1):
        for i in range(len(words) - order + 1):
            gram = tuple(words[i:i + order])
            if ngram_counts[order].get(gram) is None:
                ngram_counts[order][gram] = 1
            else:
                ngram_counts[order][gram] += 1

In [11]:
# Set the maximum n-gram order (e.g., 2 for bigrams, 3 for trigrams, etc.)
MAX_N = 3
for sentence in tqdm(train_sentences, desc="Building n-gram counts"):
    build_ngram_counts(words(sentence), MAX_N)

total_words = len(train_words)  # Total number of words in the training set
V = len(ngram_counts[1])  # Vocabulary size

Building n-gram counts:   0%|          | 0/231212 [00:00<?, ?it/s]

Building n-gram counts: 100%|██████████| 231212/231212 [00:08<00:00, 26671.51it/s]


In [12]:
WORDS['the']

258815

In [13]:
ngram_counts[1][('the', )]

258815

In [39]:
import math

# --- Functions to compute n-gram probabilities with smoothing ---

def compute_bigram_prob(w1, w2, alpha):
    """
    Computes smoothed bigram probability:
      P(w2|w1) = (C(w1, w2) + alpha) / (C(w1) + alpha * |V|)
    """
    c_bigram = ngram_counts.get(2, {}).get((w1, w2), 0)
    c_unigram = ngram_counts.get(1, {}).get((w1,), 0)
    return (c_bigram + alpha) / (c_unigram + alpha * V)

def compute_trigram_prob(w1, w2, w3, alpha):
    """
    Computes smoothed trigram probability:
      P(w3|w1,w2) = (C(w1, w2, w3) + alpha) / (C(w1, w2) + alpha * |V|)
    """
    c_trigram = ngram_counts.get(3, {}).get((w1, w2, w3), 0)
    c_bigram = ngram_counts.get(2, {}).get((w1, w2), 0)
    return (c_trigram + alpha) / (c_bigram + alpha * V)

def compute_interpolated_prob(w1, w2, w3, alpha, lam):
    """
    Computes the interpolated probability:
      P(w3|w1,w2) = lam * P_trigram(w3|w1,w2) + (1 - lam) * P_bigram(w3|w2)
    where the bigram probability is computed as:
      P(w3|w2) = (C(w2, w3) + alpha) / (C(w2) + alpha * |V|)
    """
    # Trigram probability
    p_tri = compute_trigram_prob(w1, w2, w3, alpha)
    
    # Bigram probability: note we use context w2 only
    c_bigram = ngram_counts.get(2, {}).get((w2, w3), 0)
    c_unigram = ngram_counts.get(1, {}).get((w2,), 0)
    p_bi = (c_bigram + alpha) / (c_unigram + alpha * V)
    
    return lam * p_tri + (1 - lam) * p_bi

# --- Functions to compute Cross Entropy and Perplexity ---

def compute_cross_entropy_and_perplexity(sentences, model='interpolated', alpha=0.1, lam=0.5):
    """
    Computes cross entropy and perplexity for a list of sentences.
    
    Parameters:
      sentences: list of sentences (strings).
      model: 'bigram', 'trigram', or 'interpolated'.
      alpha: smoothing parameter.
      lam: interpolation weight (only used if model=='interpolated').
      
    Returns:
      cross_entropy, perplexity
    """
    total_log_prob = 0.0
    count = 0
    for sentence in sentences:
        tokens = sentence.lower().split()
        if model == 'bigram':
            for i in range(1, len(tokens)):
                prob = compute_bigram_prob(tokens[i-1], tokens[i], alpha)
                total_log_prob += math.log2(prob)
                count += 1
        elif model == 'trigram':
            for i in range(2, len(tokens)):
                prob = compute_trigram_prob(tokens[i-2], tokens[i-1], tokens[i], alpha)
                total_log_prob += math.log2(prob)
                count += 1
        elif model == 'interpolated':
            for i in range(2, len(tokens)):
                prob = compute_interpolated_prob(tokens[i-2], tokens[i-1], tokens[i], alpha, lam)
                total_log_prob += math.log2(prob)
                count += 1
    cross_entropy = - total_log_prob / count if count > 0 else float('inf')
    perplexity = 2 ** cross_entropy
    return cross_entropy, perplexity

# --- Hyperparameter Tuning via Grid Search ---

# Use a held-out validation set (here, we take the first 1000 test sentences as an example)
held_out = test_sentences[:10000]  # Adjust this slice based on your data

# Define candidate values for alpha and lambda.
alpha_values = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1.0]
lam_values = [0.0001, 0.001, 0.01, 0.25, 0.5, 0.75, 1.0]

best_bigram_params=None
best_bigram_ce = float('inf')
print("Tuning hyperparameters for the bigram LM:")
for alpha in alpha_values:
    ce, ppl = compute_cross_entropy_and_perplexity(held_out, model='bigram', alpha=alpha)
    # print(f"Alpha: {alpha}, Cross Entropy: {ce:.4f}, Perplexity: {ppl:.4f}")
    if ce < best_bigram_ce:
        best_bigram_ce = ce
        best_bigram_params = (alpha,)
print("\nBest bigram hyperparameters (alpha, ):", best_bigram_params, "with Cross Entropy:", best_bigram_ce)

best_trigram_params=None
best_trigram_ce = float('inf')
print("Tuning hyperparameters for the trigram LM:")
for alpha in alpha_values:
    ce, ppl = compute_cross_entropy_and_perplexity(held_out, model='trigram', alpha=alpha)
    # print(f"Alpha: {alpha}, Cross Entropy: {ce:.4f}, Perplexity: {ppl:.4f}")
    if ce < best_trigram_ce:
        best_trigram_ce = ce
        best_trigram_params = (alpha,)
print("\nBest trigram hyperparameters (alpha, ):", best_trigram_params, "with Cross Entropy:", best_trigram_ce)

best_interpolated_params = None
best_interpolated_ce = float('inf')
print("Tuning hyperparameters for the interpolated LM:")
for alpha in alpha_values:
    for lam in lam_values:
        ce, ppl = compute_cross_entropy_and_perplexity(held_out, model='interpolated', alpha=alpha, lam=lam)
        # print(f"Alpha: {alpha}, Lambda: {lam}, Cross Entropy: {ce:.4f}, Perplexity: {ppl:.4f}")
        if ce < best_interpolated_ce:
            best_interpolated_ce = ce
            best_interpolated_params = (alpha, lam)

print("\nBest interpolated hyperparameters (alpha, lambda):", best_interpolated_params, "with Cross Entropy:", best_interpolated_ce)

Tuning hyperparameters for the bigram LM:

Best bigram hyperparameters (alpha, ): (0.01,) with Cross Entropy: 11.374500801714818
Tuning hyperparameters for the trigram LM:

Best trigram hyperparameters (alpha, ): (0.001,) with Cross Entropy: 13.13564369458284
Tuning hyperparameters for the interpolated LM:

Best interpolated hyperparameters (alpha, lambda): (0.001, 0.5) with Cross Entropy: 11.008184862697393


In [40]:
best_alpha = best_interpolated_params[0]
best_lambda = best_interpolated_params[1]

@lru_cache(maxsize=None)
def get_corrections(word):
    # Return the list of candidate corrections.
    # First try the word itself, then one-edit-away words.
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def my_correction(sentence, beam_width=3, n=2, alpha=best_alpha, lam=best_lambda):
    """
    Corrects a sentence using an n-gram language model with tuned hyperparameters.
    
    Parameters:
      - sentence: the input sentence to be corrected.
      - beam_width: the beam width for search.
      - n: the n-gram order to use (e.g., n=2 for bigrams, n=3 for trigrams).
      - alpha: smoothing parameter (tuned).
      - lam: interpolation weight for trigram and bigram probabilities (tuned).
    
    For each word, we use a context of up to (n-1) preceding words.
    When there isn’t enough context (at the start), we back off to lower-order models.
    """
    sentence_words = sentence.lower().split()
    candidates_seq = [(0.0, [])]
    
    for i, word in enumerate(sentence_words):
        new_candidates = []
        candidate_list = get_corrections(word)
        
        for candidate in candidate_list:
            for score, seq in candidates_seq:
                context_length = min(n - 1, len(seq))
                if context_length == 0:
                    # Use unigram probability with smoothing using alpha.
                    prob = (ngram_counts[1].get((candidate,), 0) + alpha) / (total_words + alpha * V)
                elif context_length == 1:
                    # Use bigram probability with smoothing.
                    context = tuple(seq[-1:])
                    prob = (ngram_counts.get(2, {}).get(context + (candidate,), 0) + alpha) / (
                           ngram_counts.get(1, {}).get(context, 0) + alpha * V)
                elif context_length == 2:
                    # Use interpolated probability for trigram: 
                    # P(candidate|w1,w2) = lam * trigram + (1-lam) * bigram
                    w1, w2 = seq[-2], seq[-1]
                    trigram_prob = (ngram_counts.get(3, {}).get((w1, w2, candidate), 0) + alpha) / (
                                   ngram_counts.get(2, {}).get((w1, w2), 0) + alpha * V)
                    bigram_prob = (ngram_counts.get(2, {}).get((w2, candidate), 0) + alpha) / (
                                  ngram_counts.get(1, {}).get((w2,), 0) + alpha * V)
                    prob = lam * trigram_prob + (1 - lam) * bigram_prob
                else:
                    # For contexts longer than 2, fall back to the standard n-gram formula with smoothing.
                    context = tuple(seq[-context_length:])
                    prob = (ngram_counts.get(context_length + 1, {}).get(context + (candidate,), 0) + alpha) / (
                           ngram_counts.get(context_length, {}).get(context, 0) + alpha * V)
                
                new_score = score + math.log(prob)
                new_seq = seq + [candidate]
                heapq.heappush(new_candidates, (new_score, new_seq))
                if len(new_candidates) > beam_width:
                    heapq.heappop(new_candidates)
        
        candidates_seq = new_candidates
    
    best_score, best_seq = max(candidates_seq, key=lambda x: x[0])
    return ' '.join(best_seq)


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

here

In [41]:
def add_noise(word, noise_prob=0.2):
    if random.random() > noise_prob or len(word) < 1:
        return word
    
    edits = []
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # Generate possible single edits
    if len(word) > 1:
        edits += [L + R[1:] for L, R in splits if R]  # deletes
    if len(word) > 1:
        edits += [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]  # transposes
    edits += [L + c + R[1:] for L, R in splits if R for c in letters if c != R[0]]  # replaces
    edits += [L + c + R for L, R in splits for c in letters]  # inserts
    
    return random.choice(edits) if edits else word

# Generate test data with noise
test_data = []
for sentence in test_sentences:
    original = words(sentence)
    noisy = [add_noise(word, 0.2) for word in original]
    test_data.append((original, noisy))

### Norvigs metrics on synhtetic data

In [17]:
# Evaluation metrics
norvig_correct, total = 0, 0

In [18]:
for original, noisy in tqdm(test_data, desc="Evaluating"):
    noisy_sentence = ' '.join(noisy)
    norvig_result = norvigs_correction(noisy_sentence).split()
    # Validate lengths
    if len(norvig_result) != len(original):
        continue

    # Compare results
    for o, n in zip(original, norvig_result):
        total += 1
        norvig_correct += (n == o)

Evaluating: 100%|██████████| 12170/12170 [01:22<00:00, 148.29it/s]


In [20]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# results = []

# # Hyperparameter ranges
# n_values = range(1, 7)
# beam_widths = range(1, 6)

# # Tuning loop
# for n in n_values:
#     for beam_width in beam_widths:
#         print(f"\nTesting n={n}, beam_width={beam_width}")
#         correct = 0
#         total = 0
        
#         for original, noisy in tqdm(test_data, desc=f"n={n}, bw={beam_width}"):
#             try:
#                 corrected = my_correction(' '.join(noisy), 
#                                         beam_width=beam_width, 
#                                         n=n).split()
#             except Exception as e:
#                 print(f"Error with n={n}, bw={beam_width}: {str(e)}")
#                 continue
                
#             if len(corrected) != len(original):
#                 continue
                
#             for orig_word, corrected_word in zip(original, corrected):
#                 total += 1
#                 correct += (corrected_word == orig_word)
        
#         accuracy = correct / total if total > 0 else 0
#         results.append({'n': n, 'beam_width': beam_width, 'accuracy': accuracy})
#         print(f"Accuracy: {accuracy:.4f}")

# # Save results to DataFrame
# results_df = pd.DataFrame(results)

# # Save to CSV
# results_df.to_csv('hyperparameter_results.csv', index=False)

In [21]:
# # ======= LOAD AND MERGE BIGRAMS FROM FILE =======
# bigram_file = "useful data/bigrams (2).txt"

# with open(bigram_file, encoding="latin-1") as f:
#     for line in f:
#         # Split by tabs (or adjust the delimiter if needed)
#         parts = line.strip().split("\t")
#         # Skip lines that don't have at least 3 parts (e.g., header or malformed lines)
#         if len(parts) < 3:
#             continue
#         count_str, first_word, second_word = parts[:3]
#         try:
#             count = int(count_str)
#         except ValueError:
#             continue  # Skip lines where count is not an integer
        
#         # Create the bigram tuple
#         bigram = (first_word, second_word)
        
#         # Merge: add to existing count if the bigram is already present.
#         if bigram in ngram_counts.get(2, {}):
#             ngram_counts[2][bigram] += count
#         else:
#             # Ensure that the bigram dictionary exists
#             if 2 not in ngram_counts:
#                 ngram_counts[2] = {}
#             ngram_counts[2][bigram] = count


# # Update vocabulary statistics
# V = len(ngram_counts[1])  # Update vocabulary size after adding new words

In [74]:
my_correct = 0

for original, noisy in tqdm(test_data, desc="Evaluating"):
    noisy_sentence = ' '.join(noisy)
    
    my_result = my_correction(noisy_sentence, n=3, beam_width=5).split()
    
    # Validate lengths
    if len(my_result) != len(original):
        continue
    
    # Compare results
    for o, m in zip(original, my_result):
        my_correct += (m == o)

Evaluating:   0%|          | 0/12170 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 12170/12170 [00:10<00:00, 1134.39it/s]


In [75]:
# Calculate metrics
norvig_acc = norvig_correct / total
my_acc = my_correct / total

print(f"\nNorvig's Accuracy: {norvig_acc:.4f}")
print(f"Your Algorithm's Accuracy: {my_acc:.4f}")
print(f"Improvement: {(my_acc - norvig_acc):.4f} ({((my_acc/norvig_acc)-1)*100:.1f}%)")


Norvig's Accuracy: 0.9430
Your Algorithm's Accuracy: 0.9450
Improvement: 0.0020 (0.2%)


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [45]:
def parse_holbrook_line(line):
    """
    Parse a line from Holbrook dataset, extracting:
    - Original sentence with errors
    - Correct sentence with targets
    - List of error positions and targets
    """
    errors = []
    raw_parts = []
    correct_parts = []
    
    # Split line into text and error segments
    pos = 0
    for match in re.finditer(r'<err targ=(.*?)>(.*?)</err>', line):
        target = match.group(1).strip('"')
        error = match.group(2)

        start, end = match.start(), match.end()
        
        # Add preceding text
        raw_parts.append(line[pos:start])
        correct_parts.append(line[pos:start])
        
        pos = end

        correct_parts.append(target)
        
        if len(error.split()) > 1 or len(target.split()) > 1:
            raw_parts.append(target)
            continue
        else:
            raw_parts.append(error)
        

        if error == target:
            continue

        # Record error position
        error_start = len(' '.join(raw_parts).split())
        errors.append(error_start-1)
    
    
    # Add remaining text
    raw_parts.append(line[pos:])
    correct_parts.append(line[pos:])
    
    # Create full sentences
    raw_sentence = ' '.join(''.join(raw_parts).split())
    correct_sentence = ' '.join(''.join(correct_parts).split())
    
    return raw_sentence, correct_sentence, errors

In [46]:
def evaluate_holbrook(test_lines, corrector_func, beam_width=None, n=None):
    """
    Evaluate spelling corrector on Holbrook dataset
    Returns:
    - Accuracy: % of errors corrected properly
    - Precision: % of corrections that were right
    - Recall: % of total errors corrected
    - F1: Harmonic mean of precision and recall
    """
    stats = defaultdict(int)
    
    for line in tqdm(test_lines, desc='Evaluating'):
        if not line.strip():
            continue
            
        # Parse line
        raw_sentence, correct_sentence, errors = parse_holbrook_line(line)
        
        if (beam_width is None) or (n is None):
            corrected_sentence = corrector_func(raw_sentence)
        else:    
            corrected_sentence = corrector_func(raw_sentence, beam_width=beam_width, n=n)
        
        stats["total_errors"] += len(errors)
        
        for i, (raw, corrected, correct) in enumerate(zip(raw_sentence.split(), corrected_sentence.split(), correct_sentence.split())):
            # Count a correction if the system changed the word from raw_sentence
            if raw != corrected:
                stats['total_proposed'] += 1

            # If the word was known to be in error and the correction is exactly right, count it as correct.
            if i in errors and corrected == correct:
                stats['correct'] += 1
    
    # Calculate metrics
    accuracy = stats['correct'] / stats['total_errors'] if stats['total_errors'] else 0
    precision = stats['correct'] / stats['total_proposed'] if stats['total_proposed'] else 0
    recall = stats['correct'] / stats['total_errors'] if stats['total_errors'] else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'total_errors': stats['total_errors'],
        'correct_corrections': stats['correct'],
        'incorrect_corrections': stats['incorrect']
    }

In [47]:
import re
# Load test data
with open('holbrook-tagged.dat.txt') as f:
    text = f.read().lower()

print(len(text))

sentences = re.split(r'(?<=[.!?])\s+', text.strip())

sentence_index = 17
print(sentences[sentence_index])

parse_holbrook_line(sentences[sentence_index])

186633
we have got anglia like to <err targ=watch> wach </err> <err targ=cowboys> cow boys </err> .


('we have got anglia like to wach cowboys .',
 'we have got anglia like to watch cowboys .',
 [6])

## Norvig's solution Metrics

In [48]:
# Run evaluation
metrics = evaluate_holbrook(sentences, norvigs_correction)

print(f"Accuracy: {metrics['accuracy']:.2%}")
print(f"Precision: {metrics['precision']:.2%}")
print(f"Recall: {metrics['recall']:.2%}")
print(f"F1 Score: {metrics['f1']:.2%}")

Evaluating: 100%|██████████| 1057/1057 [00:36<00:00, 28.92it/s]

Accuracy: 17.80%
Precision: 13.57%
Recall: 17.80%
F1 Score: 15.40%


## My solution metrics

In [49]:
# Run evaluation
metrics = evaluate_holbrook(sentences, my_correction, n=3, beam_width=5)

print(f"Accuracy: {metrics['accuracy']:.2%}")
print(f"Precision: {metrics['precision']:.2%}")
print(f"Recall: {metrics['recall']:.2%}")
print(f"F1 Score: {metrics['f1']:.2%}")

Evaluating: 100%|██████████| 1057/1057 [00:04<00:00, 235.77it/s]

Accuracy: 20.12%
Precision: 15.34%
Recall: 20.12%
F1 Score: 17.41%


##  Word Error Rate (WER)

WER is a metric used to evaluate the performance of systems that generate or correct sequences of words, such as automatic speech recognition systems or spelling correction systems. It measures the number of word-level errors made by a system when comparing its output to the reference or ground truth text.

### $ WER = \frac{S + I + D}{N} $

* $ S $ : the number of substitutions
* $ i $ : the number of insertions
* $ D $ : the number of deletions
* N: the total number of words in the reference

In [60]:
def parse_holbrook_line_without_indexes(line):
    """
    Parse a line from Holbrook dataset, extracting:
    - Original sentence with errors
    - Correct sentence with targets
    - List of error positions and targets
    """
    raw_parts = []
    correct_parts = []
    
    # Split line into text and error segments
    pos = 0
    for match in re.finditer(r'<err targ=(.*?)>(.*?)</err>', line):
        target = match.group(1).strip('"')
        error = match.group(2)

        start, end = match.start(), match.end()
        
        # Add preceding text
        raw_parts.append(line[pos:start])
        correct_parts.append(line[pos:start])
        
        pos = end

        correct_parts.append(target)

        raw_parts.append(error)

        # Record error position
        error_start = len(' '.join(raw_parts).split())
    
    
    # Add remaining text
    raw_parts.append(line[pos:])
    correct_parts.append(line[pos:])
    
    # Create full sentences
    raw_sentence = ' '.join(''.join(raw_parts).split())
    correct_sentence = ' '.join(''.join(correct_parts).split())
    
    return raw_sentence, correct_sentence

In [61]:
sentence_index = 17
print(sentences[sentence_index])

parse_holbrook_line_without_indexes(sentences[sentence_index])

we have got anglia like to <err targ=watch> wach </err> <err targ=cowboys> cow boys </err> .


('we have got anglia like to wach cow boys .',
 'we have got anglia like to watch cowboys .')

In [64]:
Levenshtein =  nltk.edit_distance

def evaluate_wer(test_lines, corrector_func, beam_width=None, n=None):

    total_wer = 0
    num_sentences = len(test_lines)
    
    for line in tqdm(test_lines, desc='Evaluating'):
        if not line.strip():
            continue
            
        # Parse line
        raw_sentence, correct_sentence = parse_holbrook_line_without_indexes(line)
        
        if (beam_width is None) or (n is None):
            corrected_sentence = corrector_func(raw_sentence)
        else:    
            corrected_sentence = corrector_func(raw_sentence, beam_width=beam_width, n=n)
        
        reference_words = correct_sentence.split()
        corrected_words = corrected_sentence.split()

        S = Levenshtein(reference_words, corrected_words)
        I = max(0, len(corrected_words) - len(reference_words))
        D = max(0, len(reference_words) - len(corrected_words))

        N = max(len(reference_words), len(corrected_words))

        wer = (S + I + D) / N
        total_wer += wer

    return total_wer / num_sentences

In [65]:
metrics = evaluate_wer(sentences, norvigs_correction)
metrics

Evaluating: 100%|██████████| 1057/1057 [00:34<00:00, 30.40it/s]


0.2265135868085441

In [66]:
metrics = evaluate_wer(sentences, my_correction, n=3, beam_width=5)
metrics

Evaluating: 100%|██████████| 1057/1057 [00:06<00:00, 162.12it/s]


0.22391519366337606

## Character Error Rate (CER)
CER is similar to WER but operates at the character level. It measures the accuracy of character-level transcriptions or corrections. CER quantifies the number of character-level errors made by a system when comparing its output to the reference text. Errors include substitutions, insertions, and deletions of individual characters.

### $ CER = \frac{S + I + D}{N} $

* $ S $ : the number of character substitutions
* $ i $ : the number of character insertions
* $ D $ : the number of character deletions
* N: the total number of characterσ in the reference

In [67]:
def evaluate_cer(test_lines, corrector_func, beam_width=None, n=None):

    total_cer = 0
    num_sentences = len(test_lines)
    
    for line in tqdm(test_lines, desc='Evaluating'):
        if not line.strip():
            continue
            
        # Parse line
        raw_sentence, correct_sentence = parse_holbrook_line_without_indexes(line)
        
        if (beam_width is None) or (n is None):
            corrected_sentence = corrector_func(raw_sentence)
        else:    
            corrected_sentence = corrector_func(raw_sentence, beam_width=beam_width, n=n)
        

        S = Levenshtein(correct_sentence, corrected_sentence)
        I = max(0, len(corrected_sentence) - len(correct_sentence))
        D = max(0, len(correct_sentence) - len(corrected_sentence))

        N = max(len(correct_sentence), len(corrected_sentence))

        cer = (S + I + D) / N
        total_cer += cer

    return total_cer / num_sentences

In [68]:
metrics = evaluate_wer(sentences, norvigs_correction)
metrics

Evaluating: 100%|██████████| 1057/1057 [00:36<00:00, 29.11it/s]


0.2265135868085441

In [73]:
metrics = evaluate_wer(sentences, my_correction, n=3, beam_width=7)
metrics

Evaluating: 100%|██████████| 1057/1057 [00:01<00:00, 959.98it/s] 


0.22375511025237368

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)